In [1]:
######## # AUTO LOGIN-1 #########
account_name = 'haj'
import pandas as pd
import numpy as np
import threading
import tqdm
pd.set_option('plotting.backend', 'pandas_bokeh')
import pandas_bokeh
pandas_bokeh.output_notebook()
%load_ext line_profiler

Loading BokehJS ...

In [2]:
qty_m_spot = 200

threshold = 35
qty_factor = 15
ma = 10
srnum = 24
inst_token = 299011

In [3]:
tkndf = pd.read_csv('./data_records/tokens_list_df.csv')
tkn_list = tkndf['0'].to_list()

In [4]:
class PnLCalculator:
    def __init__(self):
        self.quantity = 0
        self.cost = 0.0
        self.market_value = 0.0
        self.r_pnl = 0.0
        self.average_price = 0.0
        self.date = 0.0

    def fill(self, n_pos, exec_price,date):
        pos_change = n_pos - self.quantity
        direction = np.sign(pos_change)
        prev_direction = np.sign(self.quantity)
        qty_closing = min(abs(self.quantity), abs(pos_change)) * direction if prev_direction != direction else 0
        qty_opening = pos_change if prev_direction == direction else pos_change - qty_closing

        new_cost = self.cost + qty_opening * exec_price
        if self.quantity != 0:
            new_cost += qty_closing * self.cost / self.quantity
            self.r_pnl += qty_closing * (self.cost / self.quantity - exec_price)

        self.quantity = n_pos
        self.cost = new_cost
        self.date = date

    def update(self, price):
        if self.quantity != 0:
            self.average_price = self.cost / self.quantity
        else: 
            self.average_price = 0
        self.market_value = self.quantity * price
        return self.market_value - self.cost

In [5]:
def export_calc(log_data_lst):
    npdsf = pd.DataFrame(log_data_lst)
#     npdsf = pd.concat([depth_data_old,npdsf])
    ndf =  npdsf[npdsf.trend != 0]
    counter = 0
    positions_call = ndf['my_qty']
    exec_prices_call = ndf['ltp']
    bkg_and_charges = 64.5
    date_call = ndf['date']
    pnls_call = []
    pos_call = PnLCalculator()
    for (p,e,dt) in zip(positions_call, exec_prices_call,date_call):
        pos_call.fill(p, e,dt)
        u_pnl = pos_call.update(e)
        pnls_call.append({'date':pos_call.date,
                          'Qty':int(pos_call.quantity),
                          'r_pnl':round((1000*pos_call.r_pnl),4),
                          'u_pnl':round(1000*u_pnl,4),
                          'total_pnl':round(1000*(pos_call.r_pnl+u_pnl)-(counter*(bkg_and_charges/2)),4),
                          'average_price':round(pos_call.average_price,4),
                         })
        counter = counter +1

    pnls_df_call = pd.DataFrame(pnls_call)
    return pnls_df_call
    # print(pnls_df_call)
    # pnls_df_call['total_pnl'].plot()
    
    
#     npdsf.to_excel('./depth_data_8_1__'+str(srnum)+'.xlsx')
#     pnls_df_call.to_excel('./pnls_df_call_4_1__'+str(srnum)+'.xlsx')
    
def Average(lst): 
    return sum(lst) / len(lst) 

In [6]:
def check_nan(num):
    is_nan = False
    if(np.isneginf(num) or np.isneginf(num) or np.isinf(num) or (num != num)):
        is_nan = True
    return is_nan

In [7]:
inst_wise_analysis_list = []

In [8]:
def analyze_one_inst(one_tkn):
    global inst_wise_analysis_list
    depth_data_old = pd.read_excel(f'./data_records/instwise/{one_tkn}.xlsx')
#     del depth_data_old['Unnamed: 0.1']
#     del depth_data_old['Unnamed: 0']
    # my_qty = depth_data_old['my_qty'][depth_data_old.index.values[-1]]
    my_qty = 0
    depth_data_old


    buy_qty_lst = []
    sell_qty_lst = []
    percent_lst = []
    percent_diff_lst = []
    log_data_lst = []
    

    for i in range(len(depth_data_old.index)):

        buy_quantity = depth_data_old.loc[ i , : ]['buy_quantity']
        sell_quantity = depth_data_old.loc[ i , : ]['sell_quantity']
        diff = buy_quantity - sell_quantity
        mean = (buy_quantity + sell_quantity)/2
        if(mean==0):
            mean = 0.000001
        percent  = (diff/mean)*100
        percent_lst.append(percent)
        if(len(percent_lst)>ma):
            percent_lst.pop(0)

        percent_10ma = Average(percent_lst)
        
        if(percent_10ma==0):
            percent_10ma=0.000001
        
        percent_diff = 100*(percent-percent_10ma)/percent_10ma


        percent_diff_lst.append(percent_diff)
        if(len(percent_diff_lst)>ma):
            percent_diff_lst.pop(0)
        percent_diff_10ma = Average(percent_diff_lst)

        trend = 0
        
        if(len(percent_diff_lst)==ma):
            if(percent_diff_10ma>0 and percent_diff>threshold):
                trend = 1
                if(not check_nan(qty_m_spot*(percent_diff/100)/qty_factor)):
                    my_qty = my_qty + abs(int(qty_m_spot*(percent_diff/100)/qty_factor))

            if(percent_diff_10ma<0 and percent_diff<-threshold):
                trend = -1
                if(not check_nan(qty_m_spot*(percent_diff/100)/qty_factor)):
                    my_qty = my_qty - abs(int(qty_m_spot*(percent_diff/100)/qty_factor))


        log_data_lst.append({'buy_quantity':buy_quantity,
                             'sell_quantity':sell_quantity,
                             'diff':diff,
                             'mean':mean,
                             'percent':percent,
                             'percent_10ma':percent_10ma,
                             'percent_diff':percent_diff,
                             'percent_diff_10ma':percent_diff_10ma,
                             'trend':trend,
                             'my_qty':my_qty,
                             'ltp':depth_data_old.loc[ i , : ]['last_price'],
                             'date':depth_data_old.loc[ i , : ]['last_trade_time']})
        
    pnls_df = export_calc(log_data_lst)
    
    if(pnls_df.empty == True):
        return
    
    
    pnls_df.to_excel(f'./data_records/pnls_df_rev/pnls_df_{one_tkn}.xlsx')

    final_pnl = pnls_df.iloc[-1:]['total_pnl'].to_list()[0]
    print(f'final_pnl : {int(final_pnl)}')

    min_qty = pnls_df.describe()['Qty']['min']
    max_qty = pnls_df.describe()['Qty']['max']

    mean_price = pnls_df.describe()['average_price']['mean']

    qty_diff = max_qty - min_qty

    max_required_fund = qty_diff*mean_price
    print(f'max_required_fund : {int(max_required_fund)}')

    sharpe_ratio = np.sqrt(252)* pnls_df.describe()['total_pnl']['mean']/pnls_df.describe()['total_pnl']['std']
    print(f'sharpe_ratio : {sharpe_ratio}')

    pnl_by_max_req_fund = final_pnl/max_required_fund
    print(f'pnl_by_max_req_fund : {pnl_by_max_req_fund}')

    print(f'insttkn : {one_tkn}')
    print('=======================================')

    inst_wise_analysis_list.append({'insttkn':one_tkn,
                                    'final_pnl':final_pnl,
                                    'max_required_fund':max_required_fund,
                                    'sharpe_ratio':sharpe_ratio,
                                    'pnl_by_max_req_fund':pnl_by_max_req_fund,}) 
    
    

In [9]:
for one_tkn in tqdm.tqdm(tkn_list):
    analyze_one_inst(one_tkn)
    
#         ps_thr = threading.Thread(target=analyze_one_inst, args=([one_tkn]))
#         ps_thr.start()

  2%|▏         | 1/62 [00:22<23:21, 22.98s/it]

final_pnl : 1772432
max_required_fund : 624017
sharpe_ratio : 5.410538012673001
pnl_by_max_req_fund : 2.840356104861562
insttkn : 4267265


  3%|▎         | 2/62 [00:50<25:43, 25.73s/it]

final_pnl : 24731004
max_required_fund : 4994739
sharpe_ratio : -7.720858512468265
pnl_by_max_req_fund : 4.951410010652867
insttkn : 3001089


  5%|▍         | 3/62 [01:17<25:47, 26.23s/it]

...

final_pnl : -2644137
max_required_fund : 234130308767
sharpe_ratio : -4.3231720429748925
pnl_by_max_req_fund : -1.1293444806099145e-05
insttkn : 415745


 98%|█████████▊| 61/62 [23:05<00:23, 23.47s/it]

final_pnl : -510874600
max_required_fund : 2010637897682
sharpe_ratio : -11.344102029102292
pnl_by_max_req_fund : -0.000254085830839518
insttkn : 519937


100%|██████████| 62/62 [23:26<00:00, 22.68s/it]

final_pnl : 2503394
max_required_fund : 266898167903
sharpe_ratio : 17.405896311724177
pnl_by_max_req_fund : 9.379584862512141e-06
insttkn : 2977281


In [10]:
inst_wise_analysis_df = pd.DataFrame(inst_wise_analysis_list)
inst_wise_analysis_df.to_excel('./inst_wise_analysis_df_reverse.xlsx',index=False)

In [11]:
# %lprun -f analyze_one_inst analyze_one_inst(inst_token)

In [12]:
# pnls_df['total_pnl'].plot()

In [13]:
# 6.45(q/1) -> 1
# 7(q/1)    -> 2
# 10(q/1)   -> 3
# 13(q/1)   -> 4
# 17(q/1)   -> 5
# 25(q/1)   -> 6
# 35(q/1)   -> 7
# 45(q/1)   -> 8
# 55(q/1)   -> 9
# 35(q/10)  -> 10
# 30(q/10)  -> 11
# 35(q/12)  -> 12
# 35(q/13)  -> 13
#########################################
# 35(q/15)  -> 14 ### SELECTED CANDIDATE#
#########################################

In [14]:
# master_df_list = []
# for one_ip_tkn in tqdm.tqdm(tkn_list):
    
#     globals()[f'pnls_df_{one_ip_tkn}'] = pd.read_excel(f'./data_records/pnls_df_rev/pnls_df_{one_ip_tkn}.xlsx')
#     globals()[f'pnls_df_{one_ip_tkn}']=globals()[f'pnls_df_{one_ip_tkn}'].drop_duplicates(subset = 'date').set_index('date')['total_pnl']
#     master_df_list.append(globals()[f'pnls_df_{one_ip_tkn}'])
#     globals()[f'pnls_df_{one_ip_tkn}'].plot()

In [15]:

# # dfs = [df.set_index('date')['total_pnl'] for df in master_df_list]
# merged_df =  pd.concat(master_df_list, axis=1,)
# merged_df

In [16]:
# from functools import reduce
# dfm = reduce(lambda df1,df2: pd.merge(df1,df2,on='date', how='outer'), master_df_list)

In [17]:
# master_df[inst_token] = globals()[f'pnls_df_{inst_token}']['total_pnl']
# master_df

In [18]:
# globals()[f'pnls_df_{inst_token}']

In [19]:
# globals()[f'pnls_df_{inst_token}'][['total_pnl','date']].plot(x='date')